In [1]:
import preprocessor as p
import numpy as np
print("STARTED RUNNING THIS NOTEBOOK")

STARTED RUNNING THIS NOTEBOOK


In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm_notebook as tqdm
import preprocessor as p
import numpy as np
import matplotlib.pyplot as plt
import random
from collections import Counter
import spacy
from tqdm import tqdm, tqdm_notebook, tnrange
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

In [3]:
import pickle as pkl
from collections import defaultdict
import pandas as pd
import os
import numpy as np
import json
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, accuracy_score

In [4]:
from collections import Counter
import spacy
from tqdm import tqdm, tqdm_notebook, tnrange
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

In [5]:
import matplotlib.pyplot as plt
import random

In [6]:
class Node:
    def __init__(self,uid,tid,time_stamp,label):
        self.children = {}
        self.childrenList = []
        self.num_children = 0
        self.tid = tid
        self.uid = uid
        self.label = label
        self.time_stamp = time_stamp
    
    def add_child(self,node):
        if node.uid not in self.children:
            self.children[node.uid] = node
            self.num_children += 1
        else:
            self.children[node.uid] = node
        self.childrenList = list(self.children.values())

In [7]:
class Tree:
    def __init__(self,root):
        self.root = root
        self.tweet_id = root.tid
        self.uid = root.uid
        self.height = 0
        self.nodes = 0
    
    def show(self):
        queue = [self.root,0]
        
        while len(queue) != 0:
            toprint = queue.pop(0)
            if toprint == 0:
                print('\n')
            else:
                print(toprint.uid,end=' ')
                queue += toprint.children.values()
                queue.append(0)
                
    def insertnode(self,curnode,parent,child):
        if curnode.uid == parent.uid:
            curnode.add_child(child)
            return 1

        elif parent.uid in curnode.children:
            s = self.insertnode(curnode.children[parent.uid],parent,child)
            return 2
        else:
            for node in curnode.children:
                s = self.insertnode(curnode.children[node],parent,child)
                if s == 2:
                    break

In [8]:
def loadPklFileNum(datapath,incSize,fileNum):
    
    with open(datapath+str(incSize)+'inc_'+str(fileNum)+'.pickle', 'rb') as handle:
        twitTrees = pkl.load(handle)
    return twitTrees

In [9]:
def loadTreeFilesOfIncrement(datapath,incSize):
    twittertrees = {}
    
    files = [x for x in os.listdir(t15Datapath) if str(incSize)+'inc' in x]
    
    for file in tqdm(files):
        with open(datapath+file,'rb') as handle:
            partialTrees = pkl.load(handle)
        twittertrees.update(partialTrees)
        
    return twittertrees

In [10]:
dataset = 'twitter15'

if dataset == 'twitter15':
    %run ../twitter15/twitter15_text_processing.ipynb
    t15Datapath = '../twitter15/pickledTrees/'
    
if dataset == 'twitter16':
    %run ../twitter16/twitter16_text_processing.ipynb
    t15Datapath = '../twitter16/pickledTrees/'
# twitter15_trees = loadPklFileNum(t15Datapath,20,1)

In [11]:
twitter15_trees = loadTreeFilesOfIncrement(t15Datapath,20)

In [12]:
if dataset == 'twitter15':
    %run ../twitter15/userdata_parser.ipynb
    with open('./kfolds_twitter15.pickle','rb') as f:
        kfold_df = pkl.load(f)
    datadir = './twitter15_kfold_results/'
    
if dataset == 'twitter16':
    %run ../twitter16/userdata_parser.ipynb
    with open('./kfolds_twitter16.pickle','rb') as f:
        kfold_df = pkl.load(f)
    datadir = './twitter16_kfold_results/'

100%|██████████| 430343/430343 [01:27<00:00, 4921.27it/s]


In [13]:
for key in tqdm(userVects):
    userVects[key] = userVects[key].float()

userVects = defaultdict(lambda:torch.tensor([1.1100e+02, 1.5000e+01, 0.0000e+00, 7.9700e+02, 4.7300e+02, 0.0000e+00,
        8.3326e+04, 1.0000e+00]),userVects)

100%|██████████| 430343/430343 [00:01<00:00, 296193.27it/s]


In [14]:
%run ./textEncoders.ipynb

I0306 14:46:22.169637 140318102980416 file_utils.py:41] PyTorch version 1.2.0 available.


In [15]:
%run ./temporal_tree_model.ipynb

In [16]:
if torch.cuda.is_available():
    device = 'cuda:1'
    device = 'cpu'
else:
    device = 'cpu'

In [17]:
# labelMap = {}
# labelCount = 0
# for label in list(twitter15_labels.values()):
#     if label not in labelMap:
#         labelMap[label] = labelCount
#         labelCount += 1
# labelMap

labelMap = {'true':0,'false':1,'unverified':2,'non-rumor':3}

In [18]:
def trainModelWithKfold(foldnum,datadir,model,modelname,numtrees):
    train_ids, valid_ids = kfold_df[foldnum]
    
    x_train = []
    x_test = []
    y_train = []
    y_test = []

    for tid in train_ids:
        x_train.append(tuple((twitter15_trees[tid],twitter15_text[tid])))
        y_train.append(labelMap[twitter15_labels[tid]])

    for tid in valid_ids:
        x_test.append(tuple((twitter15_trees[tid],twitter15_text[tid])))
        y_test.append(labelMap[twitter15_labels[tid]])
    
    criterion = torch.nn.CrossEntropyLoss()
    
    f = lambda m, n: [(i*n//m + n//(2*m)) for i in range(m)]
    
    optimizer = torch.optim.Adagrad(model.parameters(),lr=0.01)

    count = 0
    maxAcc = 0
    bestcr = 0

    train_iterwise = []
    val_iterwise = []

    for i in range(3):
        train_losses = []
        val_losses = []

        for treeSet,text in tqdm(x_train):
            tnum = 0

            idxs = f(numtrees,len(treeSet))
    #         for idx in idxs:
            trees = [ treeSet[idx] for idx in idxs ]
            count += 1
            tnum += 1
            optimizer.zero_grad()

            pred = model(trees)
            label = Variable(torch.tensor(labelMap[trees[0].root.label]))

            loss = criterion(pred.reshape(-1,4), label.reshape(-1))    

            loss.backward()
            optimizer.step()

        preds = []
        labels = []

        allLabels = []
        allPreds = []

        with torch.no_grad():
            for valSet,text in tqdm(x_test):
                idxs = f(numtrees,len(valSet))
                trees = [ valSet[idx] for idx in idxs ]

                predicted = model(trees)
                preds.append(predicted)

                predicted =  torch.softmax(predicted[0],0)
                predicted = torch.max(predicted, 0)[1].cpu().numpy().tolist()

                labels.append(labelMap[trees[0].root.label])

                allLabels.append(labelMap[trees[0].root.label])
                allPreds.append(predicted)

        predTensor = torch.stack(preds)
        labelTensor = torch.tensor(labels).to(device)

        print(allLabels,allPreds)

        loss = criterion(predTensor.reshape(-1,4), labelTensor.reshape(-1))

        cr = classification_report(allLabels,allPreds,output_dict=True)
        cr['loss'] = loss.item()
        cr['Acc'] = accuracy_score(allLabels,allPreds,)
        print('loss: ',cr['loss'])
        print(cr['Acc'])
        cr['fold'] = foldnum

        if cr['Acc'] > maxAcc:
                maxAcc = cr['Acc']
                bestcr = cr
                torch.save({'state_dict': model.state_dict()}, datadir+modelname+'_'+str(foldnum)+'.pth')

    with open(datadir+modelname+'.json', 'a') as fp:
            json.dump(bestcr, fp)
            fp.write('\n')

#     val_losses.append(loss.item())
#     train_iterwise.append(np.array(train_losses).mean())
#     val_iterwise.append(np.array(val_losses).mean())

In [19]:
criterion = torch.nn.CrossEntropyLoss()
numtreearr = [15]

for numtrees in numtreearr:
    modelname = 'decay_tempTreeEnc_'+str(numtrees)
    for i in range(1,4):
        model = temporalDecayTreeEncoder(torch.cuda.is_available(),8,100,userVects,twitter15_labels,labelMap,criterion,device)
        trainModelWithKfold(i,datadir,model,modelname,numtrees)

100%|██████████| 1/1 [00:00<00:00,  7.87it/s]
/home/gmanish/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/gmanish/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
  0%|          | 0/1 [00:00<?, ?it/s]

[2] [1]
loss:  2.253389596939087
0.0


  0%|          | 0/1 [00:00<?, ?it/s]

[1] [1]
loss:  0.7927819490432739
1.0


  0%|          | 0/1 [00:00<?, ?it/s]

...

[1] [2]
loss:  6.986588954925537
0.0


  0%|          | 0/1 [00:00<?, ?it/s]

[2] [2]
loss:  4.60137271147687e-05
1.0


100%|██████████| 1/1 [00:00<00:00,  1.56it/s]

[0] [2]
loss:  17.359285354614258
0.0


In [20]:
criterion = torch.nn.CrossEntropyLoss()
model = lstmTreeEncoder(torch.cuda.is_available(),8,100,userVects,twitter15_labels,labelMap,criterion,device)
numtreearr = [10,15]
modelname = 'std_tempTreeEnc_'+str(numtrees)

for numtrees in numtreearr:
    for i in range(0,4):
        trainModelWithKfold(i,datadir,model,modelname,numtrees)

  0%|          | 0/1 [00:00<?, ?it/s]

[2] [1]
loss:  1.8845772743225098
0.0


  0%|          | 0/1 [00:00<?, ?it/s]

[1] [1]
loss:  0.5592848062515259
1.0


  0%|          | 0/1 [00:00<?, ?it/s]

...

[1] [1]
loss:  0.07381763309240341
1.0


  0%|          | 0/1 [00:00<?, ?it/s]

[2] [1]
loss:  2.3132643699645996
0.0


100%|██████████| 1/1 [00:00<00:00,  1.68it/s]

[0] [2]
loss:  8.622230529785156
0.0


epochs = 10
X = []
y = []
X_text = []



for tid in twitter15_trees:
        if tid in twitter15_trees and tid in twitter15_labels:
            X.append(tuple((twitter15_trees[tid],twitter15_text[tid])))
            y.append(labelMap[twitter15_labels[tid]])
            X_text.append(twitter15_text[tid])
            
x_train,x_test,y_train,y_test = train_test_split(X,y,random_state=2018)

In [21]:

# model.load_state_dict(torch.load('./pretrainedModels-Twit15/'+'std_tempTreeEnc_pretrained_inc20'+'.pth')['state_dict'])

rnnTypes = ['gru','lstm']
bidirTypes = [True,False]

for rnnType in rnnTypes:
    for bidirType in bidirTypes:
        textparams[bidir] = bidirType
        textparams[rnnType] = rnnType
        
        textEncoderModel = TextEncoder('rnn',textparams,X_text,y,device)
        textEncoderModel.trainModel()
        print(len(textEncoderModel.word2idx))
        
        modelname = rnnType
        if bidirType:
            modelname = 'bidir'+modelname
        
        torch.save({'state_dict': textEncoderModel.optimalParams}, './pretrainedModels-Twit15/'+modelname+'.pth')